<br><br>
# <strong>Tensor Flow Deep Learning
<strong>Chapter 7 : 합성곱 신경망 (Convolution Neural Network)</strong> 
1. <strong>Densly(집약적) Connected Convolution Network</strong> : X를 모두 연결하고, 레이어 사이는 concat 으로 연결한다

https://www.youtube.com/watch?v=fe2Vn0mwALI
1. <strong>Dilated convolution Network</strong> : CNN 모델이 커져도 연산횟수는 동일하게 부담을 줄인다 

http://dogfoottech.tistory.com/165

<br>
## <strong>1 CNN
<strong>Convolution Neural Network</strong> : 2D convolution 예제
1. <strong>Convolution Layer : 합성곱 계층</strong>으로 가중치와 편향을 적용한다
1. <strong>Pooling Layer</strong> : 단순히 최적의 값을 선택하여 가져온다
1. <strong>Kernel</strong> : 3x3 <strong>Kernel</strong>을, 1 <strong>stride</strong> 씩 이동하며 <strong>Convolution layer</strong>를 생성

<img src="http://rnd.azoft.com/wp-content/uploads_rnd/2016/11/overall-1024x256.png" align='left' width='600'>

<br>
## <strong>2 CNN 모델의 구현
<strong>Convolution Neural Network</strong>

In [1]:
# CNN을 활용하여 인식률 높이기
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data/mnist/", one_hot=True)

# 1. 신경망 모델 구성
# 1) Tensor를 정의 : 2차원 평면과 특성치 형태를 갖는 구조로 변경
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

/home/markbaum/Python/python36/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz


In [2]:
# 2) 신경망을 정의
W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev=0.01)) # [3 3]:커널, 1: X의 특성, 32:필터
# tf.nn.conv2d() : 컨볼루션 레이어를 생성  cf) padding='SAME' : 커널 슬라이딩을 픽셀과 일치
# L1 Conv shape  : (?, 28, 28, 32)  --->   Pool : (?, 14, 14, 32)
L1 = tf.nn.conv2d(X, W1, strides=[1,1,1,1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME') # tf.nn.max_pool: Pooling구성
# L1 = tf.nn.dropout(L1, keep_prob)

In [3]:
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
# L2 Conv shape : (?, 14, 14, 64)  --->   Pool : (?, 7, 7, 64)
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
# L2 = tf.nn.dropout(L2, keep_prob)

In [4]:
# Fully Connected Layer : 직전의 Pool 사이즈 (?, 7, 7, 64)를 참고,  
# Reshape ---> (?, 256) : 입력값 7x7x64 -> 출력값 256
W3 = tf.Variable(tf.random_normal([7 * 7 * 64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7 * 7 * 64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, keep_prob)

In [5]:
# L3 에서의 출력 256개를 입력으로 받아, 0~9 레이블인 10개의 출력을 생성
W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L3, W4)

In [6]:
# 3) 손실함수를 정의
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = model, labels = Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)
# optimizer = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)   # 결과확인 가능

In [7]:
# 2. 신경망 모델 학습
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
# 1) 파라미터를 정의
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

In [8]:
# 2) 모델의 학습
for epoch in range(15):
    total_cost = 0
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1) # CNN 위한 [28 28 1] 의 형태로 재구성
        _, cost_val = sess.run([optimizer, cost],
                               feed_dict={X: batch_xs, Y: batch_ys, keep_prob: 0.7})
        total_cost += cost_val
    print('Epoch:', '%04d' % (epoch + 1),
          'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))
print('최적화 완료!')

Epoch: 0001 Avg. cost = 0.358
Epoch: 0002 Avg. cost = 0.107
Epoch: 0003 Avg. cost = 0.078
Epoch: 0004 Avg. cost = 0.061
Epoch: 0005 Avg. cost = 0.050
Epoch: 0006 Avg. cost = 0.043
Epoch: 0007 Avg. cost = 0.036
Epoch: 0008 Avg. cost = 0.032
Epoch: 0009 Avg. cost = 0.027
Epoch: 0010 Avg. cost = 0.026
Epoch: 0011 Avg. cost = 0.024
Epoch: 0012 Avg. cost = 0.021
Epoch: 0013 Avg. cost = 0.018
Epoch: 0014 Avg. cost = 0.017
Epoch: 0015 Avg. cost = 0.015
최적화 완료!


In [9]:
# 3. 모델의 성능 확인
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도:', sess.run(accuracy,
                        feed_dict={X: mnist.test.images.reshape(-1, 28, 28, 1),
                                   Y: mnist.test.labels,
                                   keep_prob: 1}))

정확도: 0.99


<br>
## <strong>3 고수준 API
<strong>Layer 모듈을 사용하여 CNN을 간단하게 구현</strong>

In [10]:
# tensorflow.layers 를 활용하여 MNIST CNN 모듈을 구현
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data/mnist/", one_hot=True)

# 1. 신경망 모델 구성
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
is_training = tf.placeholder(tf.bool)

# 기본적으로 inputs, outputs size, kernel_size 만 넣어주면
# 활성화 함수/ 컨볼루션 신경망을 위한 수치들은 자동으로 계산
# Weights 를 계산시 xavier_initializer 를 사용
L1 = tf.layers.conv2d(X, 32, [3, 3])
L1 = tf.layers.max_pooling2d(L1, [2, 2], [2, 2])
L1 = tf.layers.dropout(L1, 0.7, is_training)

Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz


In [11]:
L2 = tf.layers.conv2d(L1, 64, [3, 3])
L2 = tf.layers.max_pooling2d(L2, [2, 2], [2, 2])
L2 = tf.layers.dropout(L2, 0.7, is_training)

In [12]:
L3 = tf.contrib.layers.flatten(L2)
L3 = tf.layers.dense(L3, 256, activation=tf.nn.relu)
L3 = tf.layers.dropout(L3, 0.5, is_training)

In [13]:
model = tf.layers.dense(L3, 10, activation=None)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [14]:
# 2. 신경망 모델의 학습
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
# 1) 파라미터 설정
batch_size = 100
total_batch = int(mnist.train.num_examples/batch_size)
# 2) 모델의 학습
for epoch in range(15):
    total_cost = 0
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        _, cost_val = sess.run([optimizer, cost],
                               feed_dict={X: batch_xs,
                                          Y: batch_ys,
                                          is_training: True})
        total_cost += cost_val
    print('Epoch:', '%04d' % (epoch + 1),
          'Avg. cost =', '{:.4f}'.format(total_cost / total_batch))
print('최적화 완료!')

Epoch: 0001 Avg. cost = 0.1847
Epoch: 0002 Avg. cost = 0.0478
Epoch: 0003 Avg. cost = 0.0306
Epoch: 0004 Avg. cost = 0.0213
Epoch: 0005 Avg. cost = 0.0162
Epoch: 0006 Avg. cost = 0.0127
Epoch: 0007 Avg. cost = 0.0103
Epoch: 0008 Avg. cost = 0.0104
Epoch: 0009 Avg. cost = 0.0091
Epoch: 0010 Avg. cost = 0.0058
Epoch: 0011 Avg. cost = 0.0088
Epoch: 0012 Avg. cost = 0.0088
Epoch: 0013 Avg. cost = 0.0051
Epoch: 0014 Avg. cost = 0.0046
Epoch: 0015 Avg. cost = 0.0040
최적화 완료!


In [15]:
# 3. 훈련 모델의 성능 확인
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도:', sess.run(accuracy,
                        feed_dict={X: mnist.test.images.reshape(-1, 28, 28, 1),
                                   Y: mnist.test.labels,
                                   is_training: False}))

정확도: 0.9895
